In [10]:
%reload_ext autoreload
%autoreload 2

In [11]:
import mxnet as mx
from mxnet import nd
import mxnet.gluon as gl
import numpy as np

In [12]:
test_root = './data/ai_challenger_scene_test_a_20170922/'

In [13]:
def transform_test(img):
    if min(img.shape[0], img.shape[1]) < 224:
        img = mx.image.resize_short(img, 224)
    im_h, im_w = img.shape[0], img.shape[1]
    img1 = mx.image.fixed_crop(img, 0, 0, 224, 224)
    img2 = mx.image.fixed_crop(img, im_w-224, 0, 224, 224)
    img3 = mx.image.fixed_crop(img, 0, im_h-224, 224, 224)
    img4 = mx.image.fixed_crop(img, im_w-224, im_h-224, 224, 224)
    
    img1 = img1.astype('float32') / 255
    img2 = img2.astype('float32') / 255
    img3 = img3.astype('float32') / 255
    img4 = img4.astype('float32') / 255
    
    img1 = mx.image.color_normalize(img1, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img2 = mx.image.color_normalize(img2, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img3 = mx.image.color_normalize(img3, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    img4 = mx.image.color_normalize(img4, mean=nd.array([0.4960, 0.4781, 0.4477]), 
                             std=nd.array([0.2915, 0.2864, 0.2981]))
    
    img1 = nd.transpose(img1, (2, 0, 1))
    img2 = nd.transpose(img2, (2, 0, 1))
    img3 = nd.transpose(img3, (2, 0, 1))
    img4 = nd.transpose(img4, (2, 0, 1))
    
    return img1, img2, img3, img4

In [14]:
from data_utils import TestDataSet
from dataloader import DataLoader

In [15]:
test_img_path = test_root + '/scene_test_a_images_20170922/'
test_set = TestDataSet(test_img_path, transform_test)
test_data = DataLoader(test_set, 32, shuffle=False, last_batch='keep')

In [16]:
ctx = mx.gpu(0)
net = gl.model_zoo.vision.resnet50_v2(classes=80)
net.initialize(ctx=ctx)
# net.load_params('./net.params')
net.hybridize()

In [17]:
import json

In [18]:
sub_list = []
i = 0
for im_name, im1, im2, im3, im4 in test_data:
    im1 = im1.as_in_context(ctx)
    im2 = im2.as_in_context(ctx)
    im3 = im3.as_in_context(ctx)
    im4 = im4.as_in_context(ctx)
    
    out1 = net(im1)
    out2 = net(im2)
    out3 = net(im3)
    out4 = net(im4)
    
    out = (out1 + out2 + out3 + out4) / 4
    pred = nd.topk(out, 1, k=3)
    
    for n in range(len(im_name)):
        temp_dict = {'image_id': im_name[n], 'label_id': list(pred[n].astype('int').asnumpy())}
        sub_list.append(temp_dict)

In [19]:
with open('submission.json', 'w') as fout:
    json.dump(sub_list, fout)

[{'image_id': '9b18e94ca3a1163f5255b39aadff9be8a73dfd83.jpg',
  'label_id': [70, 23, 49]},
 {'image_id': '8ef49140ade727556a00fae5b35bb6163b00943b.jpg',
  'label_id': [49, 23, 70]},
 {'image_id': '0983a5155246d936c763f27fc660038daa98f69c.jpg',
  'label_id': [23, 49, 70]},
 {'image_id': '3813a607bd2859290ad80b5120b1f94ada1df789.jpg',
  'label_id': [23, 70, 49]},
 {'image_id': '692643da4b6a5cd1ccf9334944dddb938e5a95fb.jpg',
  'label_id': [49, 63, 23]},
 {'image_id': '7801552998e0810579324466ba7f8f22a3fd5a4f.jpg',
  'label_id': [23, 49, 70]},
 {'image_id': '5f5de3c996cc247340e8383a00ef0a7558079e4e.jpg',
  'label_id': [70, 49, 23]},
 {'image_id': '8c778ad8dcd3a82d063f7794cc5c6c5ee5d8e730.jpg',
  'label_id': [23, 49, 70]},
 {'image_id': 'd25631243010176cea0870897a68d9c556aa9dae.jpg',
  'label_id': [23, 49, 70]},
 {'image_id': '3e70cd290b15dfe79061ce7bbb06d691abfdb602.jpg',
  'label_id': [49, 23, 70]},
 {'image_id': '328294aea2270e2c8a0d89c497cda1790634377c.jpg',
  'label_id': [23, 49, 70]},